# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

ModuleNotFoundError: No module named 'tqdm'

In [72]:
filepath = './data/user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [73]:
filepath = './data/movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [74]:
from sklearn.preprocessing import LabelEncoder

In [75]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [76]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [77]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [78]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [79]:
df_movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtAllCriticsRating,rtAllCriticsNumReviews,rtAllCriticsNumFresh,...,rtAllCriticsScore,rtTopCriticsRating,rtTopCriticsNumReviews,rtTopCriticsNumFresh,rtTopCriticsNumRotten,rtTopCriticsScore,rtAudienceRating,rtAudienceNumRatings,rtAudienceScore,rtPictureURL
0,0,Toy story,114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,toy_story,9,73,73,...,100,8.5,17,17,0,100,3.7,102338,81,http://content7.flixster.com/movie/10/93/63/10...
1,1,Jumanji,113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995,1068044-jumanji,5.6,28,13,...,46,5.8,5,2,3,40,3.2,44587,61,http://content8.flixster.com/movie/56/79/73/56...
2,2,Grumpy Old Men,107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993,grumpy_old_men,5.9,36,24,...,66,7,6,5,1,83,3.2,10489,66,http://content6.flixster.com/movie/25/60/25602...
3,3,Waiting to Exhale,114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995,waiting_to_exhale,5.6,25,14,...,56,5.5,11,5,6,45,3.3,5666,79,http://content9.flixster.com/movie/10/94/17/10...
4,4,Father of the Bride Part II,113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995,father_of_the_bride_part_ii,5.3,19,9,...,47,5.4,5,1,4,20,3,13761,64,http://content8.flixster.com/movie/25/54/25542...


In [80]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


## Матрица рейтингов

In [10]:
from scipy.sparse import coo_matrix, csr_matrix

In [84]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [85]:
R.toarray()

array([[0. , 0. , 1. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [4.5, 0. , 2. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ]])

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [86]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [14]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [15]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [101]:
user_1_rated = (user_1 != 0)
user_1_rated.toarray()

array([[False, False,  True, ..., False, False, False]])

Можем их "индексировать"

In [102]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [18]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [19]:
(user_1).multiply(user_2)

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [105]:
user_1.dot(user_2.T).toarray()

array([[216.75]])

И превращать разреженную матрицу (вектор) в плотную

In [21]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [109]:
def cosine_similarity_pair_users(u, v):
    c = np.logical_and(u.toarray(), v.toarray())
    if c.any():
        return u.dot(v.T)[0, 0]/(np.sqrt(u[c].dot(u[c].T)[0, 0]) * np.sqrt(v[c].dot(v[c].T)[0, 0]))
    else:
        return 0

In [207]:
def cosne_similarity_pair_users(u, v):
    c = np.logical_and(u.toarray(), v.toarray())
    if c.sum() > 2:
        return u[c].dot(v[c].T)[0, 0]/(np.sqrt(u[c].dot(u[c].T)[0, 0]) * np.sqrt(v[c].dot(v[c].T)[0, 0]))
    else:
        return 0


In [208]:
answer1 = round(cosne_similarity_pair_users(R[146], R[239]),3)
print(answer1)

0.923


In [164]:
np.logical_and(R[146].toarray(), R[239].toarray()).sum()

524

In [162]:
R[146].toarray(), R[239].toarray()

(array([[4.5, 0. , 0. , ..., 0. , 0. , 0. ]]),
 array([[4. , 2.5, 0. , ..., 0. , 0. , 0. ]]))

Введите значение answer1 на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy

## Функция нахождения пользователей, схожих с данным. 

Реализуйте функцию <font color = "blue">similar_users</font>(u, R, n_neigbours) которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей 
и возвращает отсортированный массив пользователей (сортировка по неубыванию), максимально похожих на данного. Для сортировки используйте np.argsort без параметров. (https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html) (Сам пользователь будет в этом списке на первом месте). Эту функцию вы сможете использовать далее. 

In [171]:
def similar_users(u, R, n_neigbours):
    ## Your code here
    dist = []
    for i in range(R.shape[0]):
        dist.append(cosne_similarity_pair_users(R[u],R[i]))
    ans = np.argsort(dist)
    ans = np.flip(ans)
    return (ans[:n_neigbours])
    

In [139]:
from sklearn.metrics.pairwise import cosine_similarity

In [172]:
answer2 = np.array2string(similar_users(42, R, 10)).replace(' ','').replace('[','').replace(']','')

In [183]:
answer2 


'42281633724815262065016921506'

In [184]:
similar_users(42, R, 10)

array([  42,  281,  633,  724,  815,    2,  620,  650, 1692, 1506],
      dtype=int64)

Введите значение answer2 без кавычек  на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy. Это будет строка из 29 символов, которая начинается на 42.

## Функция прогнозирования рейтинга

Реализуйте функцию <font color = "blue">rate_items_user</font>(u, R, n_neigbours), которая принимает на входе:
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей <font color = "red">(Теперь обратите внимание, несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)</font>

и возвращает вектор с предсказанными рейтингами по всем фильмам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [278]:
def rate_items_user(u, R, n_neigbours):
    predictions = csr_matrix((1, R.shape[1]))
    ## Your code here
    sim_u = similar_users(u, R, n_neigbours)
    sim_u = sim_u[(sim_u != 20)]
    R_v = R[sim_u].toarray()
    s_uv = np.array([cosne_similarity_pair_users(R[u], R[i]) for i in sim_u])
    predictions = s_uv.dot(R_v) / np.abs(s_uv).sum()
    
    return predictions

In [264]:
u,n_neigbours = 20,30
predictions = csr_matrix((1, R.shape[1]))
    ## Your code here
sim_u = similar_users(u, R, n_neigbours)
sim_u = sim_u[(sim_u != 20)]
R_v = R[sim_u].toarray()
s_uv = np.array([cosne_similarity_pair_users(R[u], R[i]) for i in sim_u])
#predictions = s_uv.dot(R_v) / np.abs(s_uv).sum
len(s_uv)

29

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).

* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

Т.е. предсказанные рейтинги можно получить так:
R_hat = <font color = "blue">rate_items_user</font>(20, R, n_neigbours=30). При сортировке фильмов по рейтингу используйте функцию <font color = "blue">argsort </font> без параметров.

### _Решение_

In [293]:
R_hat = rate_items_user(20, R, n_neigbours=30)
R_us = np.sign(R[u].toarray()[0])
print(R_us)
R_us = np.where(R_us==1, 2, R_us)
R_us = np.where(R_us==0, 1, R_us)
R_us = np.where(R_us==2, 0, R_us)
R_hat = R_hat * R_us
kek = np.argsort(R_hat)[::-1]
kek = kek[:5]

[0. 0. 0. ... 0. 0. 0.]


## Генерация ответа

In [289]:
answer3 = ', '.join(str(i) for i in kek)

In [290]:
answer3

'2614, 306, 343, 5573, 6720'

Полученную строку введите на странице https://www.coursera.org/learn/python-for-data-science/exam/fSPxW/sozdaniie-riekomiendatiel-noi-sistiemy Формат ответа - строка вида "X, X, X, X, X", где X - идентификаторы. Вводить ответ следует без кавычек.